In [2]:


import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_probability as tfp  
from tensorflow_probability import distributions as tfd
import tf_keras as keras
from multiprocessing import Pool

from datetime import datetime, timedelta
import logging
import sys
import os, getpass
import glob
import optuna
from optuna.trial import TrialState
import time
import math
import json
from pathlib import Path

/opt/anaconda3/envs/ddnn/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [12]:
distribution = 'Normal'
paramcount = {'Normal': 2,
              'StudentT': 3,
              'JSU': 4,
              'SinhArcsinh': 4,
              'NormalInverseGaussian': 4,
              'Point': None}
val_multi = 3 # int for # of re-trains - 1 corresponds to old approach
val_window = 364 // val_multi

if not os.path.exists(f'../trialfiles'):
    os.mkdir(f'../trialfiles')

INP_SIZE = 221
activations = ['sigmoid', 'relu', 'elu', 'tanh', 'softplus', 'softmax']

binopt = [True, False]

cty = 'ES'
repo_root = Path.cwd()
storeDBintmp = False

print(cty, distribution)

if cty != 'ES':
    raise ValueError('Incorrect country')
if distribution not in paramcount:
    raise ValueError('Incorrect distribution')

# read data file
data       = pd.read_csv(repo_root / "Datasets" / f"{cty}.csv", index_col=0) 
data.index = [datetime.strptime(e, '%Y-%m-%d %H:%M:%S') for e in data.index]
data = data.iloc[:4*364*24] # take the first 4 years - 1456 days


ES Normal


In [149]:
num_retrain = 3
INP_SIZE    = 221
activation  = ['sigmoid', 'relu', 'elu', 'leaky_relu', 'tanh', 'softplus', 'softmax']
binopt      = [True, False]

INIT_DATE_EXP   = date.fromisoformat('2020-01-01') # hyper-parameter tuning: training
VAL_INIT_DATE   = date.fromisoformat('2022-12-28') # hyper-parameter tuning: validation
TRAIN_END_DATE  = date.fromisoformat('2023-12-27') # hyper-parameter tuning: training
FINAL_DATE_EXP  = date.fromisoformat("2025-12-31")

train_val_days = (TRAIN_END_DATE - INIT_DATE_EXP).days
train_days  = (VAL_INIT_DATE - INIT_DATE_EXP).days
val_days   = (TRAIN_END_DATE - VAL_INIT_DATE).days
val_window = val_days // num_retrain

# eval_days
# calib_days
train_days

1092

In [162]:
activations = ['sigmoid', 'relu', 'elu', 'tanh', 'softplus', 'softmax']
activation[2]

'elu'

In [ ]:
%pip install optuna
!git clone https://github.com/bekzod-amonov/ddnn-execute-once.git
%cd ddnn-execute-once
!ls

import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_probability as tfp  
from tensorflow_probability import distributions as tfd
import tf_keras as keras
from multiprocessing import Pool

from datetime import datetime, timedelta
import logging
import sys
import os, getpass
import glob
import optuna
from optuna.trial import TrialState
import time
import math
import json
from pathlib import Path

print(f"==================== Obtaining Hyperparameters ====================")

distribution = 'Normal'
paramcount = {'Normal': 2,
              'StudentT': 3,
              'JSU': 4,
              'SinhArcsinh': 4,
              'NormalInverseGaussian': 4,
              'Point': None}
val_multi = 3 # int for # of re-trains - 1 corresponds to old approach
val_window = 364 // val_multi

if not os.path.exists(f'../trialfiles'):
    os.mkdir(f'../trialfiles')

INP_SIZE = 221
activations = ['sigmoid', 'relu', 'elu', 'tanh', 'softplus', 'softmax']

binopt = [True, False]

cty = 'ES'
repo_root = Path.cwd()
storeDBintmp = False

print(cty, distribution)

if cty != 'ES':
    raise ValueError('Incorrect country')
if distribution not in paramcount:
    raise ValueError('Incorrect distribution')

# read data file
data       = pd.read_csv(repo_root / "Datasets" / f"{cty}.csv", index_col=0) 
data.index = [datetime.strptime(e, '%Y-%m-%d %H:%M:%S') for e in data.index]
data = data.iloc[:4*364*24] # take the first 4 years - 1456 days

def objective(trial):
    
    # prepare the input/output dataframes
    Y = np.zeros((1456, 24))
    Yf = np.zeros((364, 24))
    for d in range(1456):
        Y[d, :] = data.loc[data.index[d*24:(d+1)*24], 'Price'].to_numpy()
    # Y = Y[7:, :] # skip first 7 days
    for d in range(364):
        Yf[d, :] = data.loc[data.index[(d+1092)*24:(d+1093)*24], 'Price'].to_numpy()
    # 
    # build X 
    X = np.zeros((1092+364, INP_SIZE))

    for d in range(7, 1092+364):
      
        X[d, :24] = data.loc[data.index[(d-1)*24:(d)*24], 'Price'].to_numpy() # D-1 price
        X[d, 24:48] = data.loc[data.index[(d-2)*24:(d-1)*24], 'Price'].to_numpy() # D-2 price
        X[d, 48:72] = data.loc[data.index[(d-3)*24:(d-2)*24], 'Price'].to_numpy() # D-3 price
        X[d, 72:96] = data.loc[data.index[(d-7)*24:(d-6)*24], 'Price'].to_numpy() # D-7 price
        X[d, 96:120] = data.loc[data.index[(d)*24:(d+1)*24], data.columns[1]].to_numpy() # D load forecast
        X[d, 120:144] = data.loc[data.index[(d-1)*24:(d)*24], data.columns[1]].to_numpy() # D-1 load forecast
        X[d, 144:168] = data.loc[data.index[(d-7)*24:(d-6)*24], data.columns[1]].to_numpy() # D-7 load forecast
        X[d, 168:192] = data.loc[data.index[(d)*24:(d+1)*24], data.columns[2]].to_numpy() # D RES sum forecast
        X[d, 192:216] = data.loc[data.index[(d-1)*24:(d)*24], data.columns[2]].to_numpy() # D-1 RES sum forecast
        
        # # original aouthors code
        # X[d, 216] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[3]].to_numpy() # D-2 EUA
        # X[d, 217] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[4]].to_numpy() # D-2 API2_Coal
        # X[d, 218] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[5]].to_numpy() # D-2 TTF_Gas
        # X[d, 219] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[6]].to_numpy() # D-2 Brent oil
        
        X[d, 216] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[3]].to_numpy().item()
        X[d, 217] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[4]].to_numpy().item()
        X[d, 218] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[5]].to_numpy().item()
        X[d, 219] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[6]].to_numpy().item()
        
        # authors code
        X[d, 220] = data.index[d].weekday()

    # '''
    # input feature selection
    colmask = [False] * INP_SIZE
    if trial.suggest_categorical('price_D-1', binopt):
        colmask[:24] = [True] * 24
    if trial.suggest_categorical('price_D-2', binopt):
        colmask[24:48] = [True] * 24
    if trial.suggest_categorical('price_D-3', binopt):
        colmask[48:72] = [True] * 24
    if trial.suggest_categorical('price_D-7', binopt):
        colmask[72:96] = [True] * 24
    if trial.suggest_categorical('load_D', binopt):
        colmask[96:120] = [True] * 24
    if trial.suggest_categorical('load_D-1', binopt):
        colmask[120:144] = [True] * 24
    if trial.suggest_categorical('load_D-7', binopt):
        colmask[144:168] = [True] * 24
    if trial.suggest_categorical('RES_D', binopt):
        colmask[168:192] = [True] * 24
    if trial.suggest_categorical('RES_D-1', binopt):
        colmask[192:216] = [True] * 24
    if trial.suggest_categorical('EUA', binopt):
        colmask[216] = True
    if trial.suggest_categorical('Coal', binopt):
        colmask[217] = True
    if trial.suggest_categorical('Gas', binopt):
        colmask[218] = True
    if trial.suggest_categorical('Oil', binopt):
        colmask[219] = True
    if trial.suggest_categorical('Dummy', binopt):
        colmask[220] = True
    X = X[:, colmask]
    
    # '''
    Xwhole = X.copy()
    Ywhole = Y.copy()
    Yfwhole = Yf.copy()
    metrics_sub = []
    
    for train_no in range(val_multi):
        
        start = val_window * train_no
        X = Xwhole[start:1092+start, :]
        Xf = Xwhole[1092+start:1092+start+val_window, :]
        Y = Ywhole[start:1092+start, :]
        Yf = Ywhole[1092+start:1092+start+val_window, :]
        X = X[7:1092, :]
        Y = Y[7:1092, :]
        
        # begin building a model
        inputs = keras.Input(shape=(X.shape[1],)) # <= INP_SIZE as some columns might have been turned off
        # batch normalization
        # we decided to always normalize the inputs
        batchnorm = True #trial.suggest_categorical('batch_normalization', [True, False])
        if batchnorm:
            norm = keras.layers.BatchNormalization()(inputs)
            last_layer = norm
        else:
            last_layer = inputs
        # dropout
        dropout = trial.suggest_categorical('dropout', binopt)
        if dropout:
            rate = trial.suggest_float('dropout_rate', 0, 1)
            drop = keras.layers.Dropout(rate)(last_layer)
            last_layer = drop
        # regularization of 1st hidden layer,
        #activation - output, kernel - weights/parameters of input
        regularize_h1_activation = trial.suggest_categorical('regularize_h1_activation', binopt)
        regularize_h1_kernel = trial.suggest_categorical('regularize_h1_kernel', binopt)
        h1_activation_rate = (0.0 if not regularize_h1_activation
                              else trial.suggest_float('h1_activation_rate_l1', 1e-5, 1e-2, log=True))
        h1_kernel_rate = (0.0 if not regularize_h1_kernel
                          else trial.suggest_float('h1_kernel_rate_l1', 1e-5, 1e-2, log=True))
        # define 1st hidden layer with regularization
        hidden = keras.layers.Dense(trial.suggest_int('neurons_1', 256, 512, log=False),
                                    activation=trial.suggest_categorical('activation_1', activations),
                                    # kernel_initializer='ones',
                                    kernel_regularizer=keras.regularizers.L1(h1_kernel_rate),
                                    activity_regularizer=keras.regularizers.L1(h1_activation_rate))(last_layer)
        # regularization of 2nd hidden layer,
        #activation - output, kernel - weights/parameters of input
        regularize_h2_activation = trial.suggest_categorical('regularize_h2_activation', binopt)
        regularize_h2_kernel = trial.suggest_categorical('regularize_h2_kernel', binopt)
        h2_activation_rate = (0.0 if not regularize_h2_activation
                              else trial.suggest_float('h2_activation_rate_l1', 1e-5, 1e-2, log=True))
        h2_kernel_rate = (0.0 if not regularize_h2_kernel
                          else trial.suggest_float('h2_kernel_rate_l1', 1e-5, 1e-2, log=True))
        # define 2nd hidden layer with regularization
        hidden = keras.layers.Dense(trial.suggest_int('neurons_2', 256, 512, log=False),
                                    activation=trial.suggest_categorical('activation_2', activations),
                                    # kernel_initializer='ones',
                                    kernel_regularizer=keras.regularizers.L1(h2_kernel_rate),
                                    activity_regularizer=keras.regularizers.L1(h2_activation_rate))(hidden)
        if paramcount[distribution] is None:
            outputs = tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., :24],scale=1e-3 + 3.0 * tf.nn.softplus(t[..., 24:])))(linear)
            # # authors one
            # outputs = keras.layers.Dense(24, activation='linear')(hidden)
            
            model = keras.Model(inputs=inputs, outputs=outputs)
            model.compile(optimizer=keras.optimizers.legacy.Adam(trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)),
                          # loss=lambda y, rv_y: -rv_y.log_prob(y),
                          loss='mae',
                          metrics='mae')
        else:
            # now define parameter layers with their regularization
            param_layers = []
            param_names = ["loc", "scale", "tailweight", "skewness"]
            for p in range(paramcount[distribution]):
                # regularize_param_kernel = True
                # param_kernel_rate = 0.1
                regularize_param_kernel = trial.suggest_categorical('regularize_'+param_names[p], binopt)
                param_kernel_rate = (0.0 if not regularize_param_kernel
                                     else trial.suggest_float(param_names[p]+'_rate_l1', 1e-5, 1e-2, log=True))
                param_layers.append(keras.layers.Dense(
                    24, activation='linear', # kernel_initializer='ones',
                    kernel_regularizer=keras.regularizers.L1(param_kernel_rate))(hidden))
            # concatenate the parameter layers to one
            linear = tf.keras.layers.concatenate(param_layers)
            # define outputs
            if distribution == 'Normal':
                outputs = tfp.layers.DistributionLambda(
                        lambda t: tfd.Normal(
                            loc=t[..., :24],
                            scale = 1e-3 + 3 * tf.math.softplus(t[..., 24:])))(linear)
            elif distribution == 'StudentT':
                outputs = tfp.layers.DistributionLambda(
                        lambda t: tfd.StudentT(
                            loc=t[..., :24],
                            scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                            df=1 + 3 * tf.math.softplus(t[..., 48:])))(linear)
            elif distribution == 'JSU':
                outputs = tfp.layers.DistributionLambda(
                        lambda t: tfd.JohnsonSU(
                            loc=t[..., :24],
                            scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                            tailweight= 1 + 3 * tf.math.softplus(t[..., 48:72]),
                            skewness=t[..., 72:]))(linear)
            elif distribution == 'SinhArcsinh':
                outputs = tfp.layers.DistributionLambda(
                        lambda t: tfd.SinhArcsinh(
                            loc=t[..., :24],
                            scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                            tailweight=1e-3 + 3 * tf.math.softplus(t[..., 48:72]),
                            skewness=t[..., 72:]))(linear)
            elif distribution == 'NormalInverseGaussian':
                outputs = tfp.layers.DistributionLambda(
                        lambda t: tfd.NormalInverseGaussian(
                            loc=t[..., :24],
                            scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                            tailweight=1e-3 + 3 * tf.math.softplus(t[..., 48:72]),
                            skewness=t[..., 72:]))(linear)
            else:
                raise ValueError(f'Incorrect distribution {distribution}')
            model = keras.Model(inputs = inputs, outputs=outputs)
            model.compile(optimizer=keras.optimizers.legacy.Adam(trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)),
                          loss=lambda y, rv_y: -rv_y.log_prob(y),
                          metrics='mae')
        # '''

        # define callbacks
        callbacks = [keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)]    
        model.fit(X, Y, epochs=300, validation_data=(Xf, Yf), callbacks=callbacks, batch_size=256, verbose=2)

        metrics = model.evaluate(Xf, Yf, verbose=0) # for point its a list of one [loss, MAE] as 
        metrics_sub.append(metrics[0])
        # we optimize the returned value, -1 will always take the model with best MAE
    return np.mean(metrics_sub)

#optuna.logging.get_logger('optuna').addHandler(logging.StreamHandler(sys.stdout))
optuna.logging.set_verbosity(optuna.logging.WARNING)
study_name = f'FINAL_DE_selection_prob_{distribution.lower()}'
if storeDBintmp:
    storage_name = f'sqlite:////tmp/{study_name}'
else:
    storage_name = f'sqlite:///../trialfiles/{study_name}'
# below calls to create either random sampler and default (tree parzen estimator)
# study can be resumed using a different one
# (or one process can sample randomly, the second one run the TPE sampler at the same time)
# study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, sampler=optuna.samplers.RandomSampler())
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
study.optimize(objective, n_trials=4, show_progress_bar=False)
best_params = study.best_params
print(best_params)
print(study.trials_dataframe())
if storeDBintmp:
    print('Trials DB stored in /tmp')
    
print(f"==================== Training/Evaluation on rolling window ====================")

if not os.path.exists(f'../forecasts_probNN_{distribution.lower()}'):
    os.mkdir(f'../forecasts_probNN_{distribution.lower()}')

if not os.path.exists(f'../distparams_probNN_{distribution.lower()}'):
    os.mkdir(f'../distparams_probNN_{distribution.lower()}')

if not os.path.exists(f'../trialfiles'):
    os.mkdir(f'../trialfiles')

def runoneday(inp):
    params, dayno = inp
    df = data.iloc[dayno*24:dayno*24+1456*24+24]
    # prepare the input/output dataframes
    Y = np.zeros((1456, 24))
    # Yf = np.zeros((1, 24)) # no Yf for rolling prediction
    for d in range(1456):
        Y[d, :] = df.loc[df.index[d*24:(d+1)*24], 'Price'].to_numpy()
    Y = Y[7:, :] # skip first 7 days
    # for d in range(1):
    #     Yf[d, :] = df.loc[df.index[(d+1092)*24:(d+1093)*24], 'Price'].to_numpy()
    X = np.zeros((1456+1, INP_SIZE))
    for d in range(7, 1456+1):
        X[d, :24] = df.loc[df.index[(d-1)*24:(d)*24], 'Price'].to_numpy() # D-1 price
        X[d, 24:48] = df.loc[df.index[(d-2)*24:(d-1)*24], 'Price'].to_numpy() # D-2 price
        X[d, 48:72] = df.loc[df.index[(d-3)*24:(d-2)*24], 'Price'].to_numpy() # D-3 price
        X[d, 72:96] = df.loc[df.index[(d-7)*24:(d-6)*24], 'Price'].to_numpy() # D-7 price
        X[d, 96:120] = df.loc[df.index[(d)*24:(d+1)*24], df.columns[1]].to_numpy() # D load forecast
        X[d, 120:144] = df.loc[df.index[(d-1)*24:(d)*24], df.columns[1]].to_numpy() # D-1 load forecast
        X[d, 144:168] = df.loc[df.index[(d-7)*24:(d-6)*24], df.columns[1]].to_numpy() # D-7 load forecast
        X[d, 168:192] = df.loc[df.index[(d)*24:(d+1)*24], df.columns[2]].to_numpy() # D RES sum forecast
        X[d, 192:216] = df.loc[df.index[(d-1)*24:(d)*24], df.columns[2]].to_numpy() # D-1 RES sum forecast
        # #original author's code
        # X[d, 216] = df.loc[df.index[(d-2)*24:(d-1)*24:24], df.columns[3]].to_numpy() # D-2 EUA
        # X[d, 217] = df.loc[df.index[(d-2)*24:(d-1)*24:24], df.columns[4]].to_numpy() # D-2 API2_Coal
        # X[d, 218] = df.loc[df.index[(d-2)*24:(d-1)*24:24], df.columns[5]].to_numpy() # D-2 TTF_Gas
        # X[d, 219] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[6]].to_numpy() # D-2 Brent oil
        X[d, 216] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[3]].to_numpy().item()
        X[d, 217] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[4]].to_numpy().item()
        X[d, 218] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[5]].to_numpy().item()
        X[d, 219] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[6]].to_numpy().item()
        # original author's code
        X[d, 220] = data.index[d].weekday()
    
    # '''
    # input feature selection
    colmask = [False] * INP_SIZE
    if params['price_D-1']:
        colmask[:24] = [True] * 24
    if params['price_D-2']:
        colmask[24:48] = [True] * 24
    if params['price_D-3']:
        colmask[48:72] = [True] * 24
    if params['price_D-7']:
        colmask[72:96] = [True] * 24
    if params['load_D']:
        colmask[96:120] = [True] * 24
    if params['load_D-1']:
        colmask[120:144] = [True] * 24
    if params['load_D-7']:
        colmask[144:168] = [True] * 24
    if params['RES_D']:
        colmask[168:192] = [True] * 24
    if params['RES_D-1']:
        colmask[192:216] = [True] * 24
    if params['EUA']:
        colmask[216] = True
    if params['Coal']:
        colmask[217] = True
    if params['Gas']:
        colmask[218] = True
    if params['Oil']:
        colmask[219] = True
    if params['Dummy']:
        colmask[220] = True
    X = X[:, colmask]
    # '''
    Xf = X[-1:, :]
    X = X[7:-1, :]
    # begin building a model
    inputs = keras.Input(X.shape[1]) # <= INP_SIZE as some columns might have been turned off
    # batch normalization
    batchnorm = True#params['batch_normalization'] # trial.suggest_categorical('batch_normalization', [True, False])
    if batchnorm:
        norm = keras.layers.BatchNormalization()(inputs)
        last_layer = norm
    else:
        last_layer = inputs
    # dropout
    dropout = params['dropout'] # trial.suggest_categorical('dropout', [True, False])
    if dropout:
        rate = params['dropout_rate'] # trial.suggest_float('dropout_rate', 0, 1)
        drop = keras.layers.Dropout(rate)(last_layer)
        last_layer = drop
    # regularization of 1st hidden layer, 
    #activation - output, kernel - weights/parameters of input
    regularize_h1_activation = params['regularize_h1_activation']
    regularize_h1_kernel = params['regularize_h1_kernel']
    h1_activation_rate = (0.0 if not regularize_h1_activation 
                          else params['h1_activation_rate_l1'])
    h1_kernel_rate = (0.0 if not regularize_h1_kernel 
                      else params['h1_kernel_rate_l1'])
    # define 1st hidden layer with regularization
    hidden = keras.layers.Dense(params['neurons_1'], 
                                activation=params['activation_1'],
                                # kernel_initializer='ones',
                                kernel_regularizer=keras.regularizers.L1(h1_kernel_rate),
                                activity_regularizer=keras.regularizers.L1(h1_activation_rate))(last_layer)
    # regularization of 2nd hidden layer, 
    #activation - output, kernel - weights/parameters of input
    regularize_h2_activation = params['regularize_h2_activation']
    regularize_h2_kernel = params['regularize_h2_kernel']
    h2_activation_rate = (0.0 if not regularize_h2_activation 
                          else params['h2_activation_rate_l1'])
    h2_kernel_rate = (0.0 if not regularize_h2_kernel 
                      else params['h2_kernel_rate_l1'])
    # define 2nd hidden layer with regularization
    hidden = keras.layers.Dense(params['neurons_2'], 
                                activation=params['activation_2'],
                                # kernel_initializer='ones',
                                kernel_regularizer=keras.regularizers.L1(h2_kernel_rate),
                                activity_regularizer=keras.regularizers.L1(h2_activation_rate))(hidden)
    if paramcount[distribution] is None:
        outputs = keras.layers.Dense(24, activation='linear')(hidden)
        model = keras.Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=keras.optimizers.Adam(params['learning_rate']),
                      loss='mae',
                      metrics='mae')
    else:
        # now define parameter layers with their regularization
        param_layers = []
        param_names = ["loc", "scale", "tailweight", "skewness"]
        for p in range(paramcount[distribution]):
            regularize_param_kernel = params['regularize_'+param_names[p]]
            param_kernel_rate = (0.0 if not regularize_param_kernel 
                                 else params[str(param_names[p])+'_rate_l1'])
            param_layers.append(keras.layers.Dense(
                24, activation='linear', # kernel_initializer='ones',
                kernel_regularizer=keras.regularizers.L1(param_kernel_rate))(hidden))
        # concatenate the parameter layers to one
        linear = tf.keras.layers.concatenate(param_layers)
        # define outputs
        if distribution == 'Normal':
            outputs = tfp.layers.DistributionLambda(
                    lambda t: tfd.Normal(
                        loc=t[..., :24],
                        scale = 1e-3 + 3 * tf.math.softplus(t[..., 24:])))(linear)
        elif distribution == 'StudentT':
            outputs = tfp.layers.DistributionLambda(
                    lambda t: tfd.StudentT(
                        loc=t[..., :24],
                        scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                        df=1 + 3 * tf.math.softplus(t[..., 48:])))(linear)
        elif distribution == 'JSU':
            outputs = tfp.layers.DistributionLambda(
                    lambda t: tfd.JohnsonSU(
                        loc=t[..., :24],
                        scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                        tailweight= 1 + 3 * tf.math.softplus(t[..., 48:72]),
                        skewness=t[..., 72:]))(linear)
        elif distribution == 'SinhArcsinh':
            outputs = tfp.layers.DistributionLambda(
                    lambda t: tfd.SinhArcsinh(
                        loc=t[..., :24],
                        scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                        tailweight=1e-3 + 3 * tf.math.softplus(t[..., 48:72]),
                        skewness=t[..., 72:]))(linear)
        elif distribution == 'NormalInverseGaussian':
            outputs = tfp.layers.DistributionLambda(
                    lambda t: tfd.NormalInverseGaussian(
                        loc=t[..., :24],
                        scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                        tailweight=1e-3 + 3 * tf.math.softplus(t[..., 48:72]),
                        skewness=t[..., 72:]))(linear) 
        else:
            raise ValueError(f'Incorrect distribution {distribution}')
        model = keras.Model(inputs = inputs, outputs=outputs)
        model.compile(optimizer=keras.optimizers.Adam(params['learning_rate']),
                      loss=lambda y, rv_y: -rv_y.log_prob(y),
                      metrics='mae')
    # '''
    # define callbacks
    callbacks = [keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)]
    perm = np.random.permutation(np.arange(X.shape[0]))
    VAL_DATA = .2
    trainsubset = perm[:int((1 - VAL_DATA)*len(perm))]
    valsubset = perm[int((1 - VAL_DATA)*len(perm)):]
    model.fit(X[trainsubset], Y[trainsubset], epochs=1500, validation_data=(X[valsubset], Y[valsubset]), callbacks=callbacks, batch_size=32, verbose=False)
    
    # metrics = model.evaluate(Xf, Yf) # for point its a list of one [loss, MAE]
    # we optimize the returned value, -1 will always take the model with best MAE

    # pred = model.predict(Xf)[0]
    if paramcount[distribution] is not None:
        dist = model(Xf)
        if distribution == 'Normal':
            getters = {'loc': dist.loc, 'scale': dist.scale}
        elif distribution == 'StudentT':
            getters = {'loc': dist.loc, 'scale': dist.scale, 'df': dist.df}
        elif distribution in {'JSU', 'SinhArcsinh', 'NormalInverseGaussian'}:
            getters = {'loc': dist.loc, 'scale': dist.scale, 
                       'tailweight': dist.tailweight, 'skewness': dist.skewness}
        print(getters)
        params = {k: [float(e) for e in v.numpy()[0]] for k, v in getters.items()}
        print(params)
        json.dump(params, open(os.path.join(f'../distparams_probNN_{distribution.lower()}', datetime.strftime(df.index[-24], '%Y-%m-%d')), 'w'))
        pred = model.predict(np.tile(Xf, (10000, 1)))
        predDF = pd.DataFrame(index=df.index[-24:])
        predDF['real'] = df.loc[df.index[-24:], 'Price'].to_numpy()
        predDF['forecast'] = pd.NA
        predDF.loc[predDF.index[:], 'forecast'] = pred.mean(0)
        # predDF.to_csv(os.path.join('../forecasts', datetime.strftime(df.index[-24], '%Y-%m-%d')))
        np.savetxt(os.path.join(f'../forecasts_probNN_{distribution.lower()}', datetime.strftime(df.index[-24], '%Y-%m-%d')), pred, delimiter=',', fmt='%.3f')
    else:
        predDF = pd.DataFrame(index=df.index[-24:])
        predDF['real'] = df.loc[df.index[-24:], 'Price'].to_numpy()
        predDF['forecast'] = pd.NA
        predDF.loc[predDF.index[:], 'forecast'] = model.predict(Xf)[0]
        pred = model.predict(Xf)
        np.savetxt(os.path.join(f'../forecasts_probNN_{distribution.lower()}', datetime.strftime(df.index[-24], '%Y-%m-%d')), pred, delimiter=',', fmt='%.3f')
    print(predDF)
    return predDF







   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 4.6 MB/s eta 0:00:00a 0:00:01
Cloning into 'ddnn-execute-once'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 33 (delta 9), reused 30 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 2.40 MiB | 8.38 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/ddnn-execute-once
commands.sh  Datasets  LICENSE	Python	README.md  trialfiles
==================== Obtaining Hyperparameters ====================
ES Normal
Epoch 1/300
5/5 - 2s - loss: 1068.4888 - mae: 69.5578 - val_loss: 1492.5305 - val_mae: 215.4227 - 2s/epoch - 303ms/step
Epoch 2/300
5/5 - 0s - loss: 325.3448 - mae: 69.2147 - val_loss: 665.4649 - val_mae: 215.2248 - 94ms/epoch - 19ms/step
Epoch 3/300
5/5 - 0s - loss: 167.9954 - mae: 68.8824 - val_loss: 468.0118 - val_mae: 214.9801 - 89ms/epoch - 18ms/step
Epoch 4/300
5/5 - 0s - loss: 112.3605 - mae: 68.

### Colab

In [ ]:
import os, glob

base = "/Users/bekzodamonov/Documents/GitHub/distributionalnn-execute-once/trialfiles"

print("BASE list:")
for name in sorted(os.listdir(base)):
    path = os.path.join(base, name)
    print(f"{name:35s}  {'DIR' if os.path.isdir(path) else 'FILE'}")

def is_sqlite_file(path: str) -> bool:
    if not os.path.isfile(path):
        return False
    try:
        with open(path, "rb") as f:
            header = f.read(16)
        return header.startswith(b"SQLite format 3")
    except Exception:
        return False

# 1) Find sqlite files directly under base (no extension needed)
sqlite_like = []
for name in os.listdir(base):
    p = os.path.join(base, name)
    if is_sqlite_file(p):
        sqlite_like.append(p)

print("\nSQLite files directly under BASE:")
print(sqlite_like)

# 2) Find sqlite files anywhere under base (recursive)
sqlite_like_recursive = []
for root, dirs, files in os.walk(base):
    for fn in files:
        p = os.path.join(root, fn)
        if is_sqlite_file(p):
            sqlite_like_recursive.append(p)

print("\nSQLite files under BASE (recursive):")
print(sqlite_like_recursive[:50], " ... total:", len(sqlite_like_recursive))

# 3) Now ACTUALLY list inside jsu1
trial_dir = os.path.join(base, "FINAL_DE_selection_prob_jsu1")
print("\nInside jsu1:")
print(sorted(os.listdir(trial_dir))[:100])


In [ ]:
print("TF:", tf.__version__)
print("Devices:", tf.config.list_physical_devices())
print("GPUs:", tf.config.list_physical_devices("GPU"))

## Rolling predictions & Evaluation

### setup and diagnostics

In [ ]:
# upgrade optuna storage (sqllineDB)
import optuna, os
from optuna.trial import TrialState

base = "/Users/bekzodamonov/Documents/GitHub/distributionalnn-execute-once/trialfiles"
db_files = sorted([os.path.join(base, f) for f in os.listdir(base)])

def summarize_db(db_path):
    storage_url = f"sqlite:///{db_path}"
    print("\n---", os.path.basename(db_path), "---")
    try:
        summaries = optuna.study.get_all_study_summaries(storage_url)
    except RuntimeError as e:
        print("CANNOT READ (schema mismatch). Fix with:")
        print(f"  optuna storage upgrade --storage {storage_url}")
        print("Error:", e)
        return

    if not summaries:
        print("No studies found in this DB.")
        return

    for s in summaries:
        study = optuna.load_study(study_name=s.study_name, storage=storage_url)
        n_complete = len(study.get_trials(states=(TrialState.COMPLETE,)))
        n_fail = len(study.get_trials(states=(TrialState.FAIL,)))
        n_running = len(study.get_trials(states=(TrialState.RUNNING,)))
        print(f"study_name={s.study_name} | trials={len(study.trials)} | COMPLETE={n_complete} FAIL={n_fail} RUNNING={n_running}")
        if n_complete:
            print("  best_value:", study.best_value)

for db in db_files:
    summarize_db(db)


In [ ]:
# author's original code
# Accepts arguments:
#     cty (currently only DE), default: DE
#     distribution (Normal, StudentT, JSU, SinhArcsinh and NormalInverseGaussian), default: Normal

distribution = 'Normal'
paramcount = {'Normal': 2,
              'StudentT': 3,
              'JSU': 4,
              'SinhArcsinh': 4,
              'NormalInverseGaussian': 4,
              'Point': None,
}

INP_SIZE = 221
# activations, neurons and params read from the trials info

cty = 'DE'

if not os.path.exists(f'../forecasts_probNN_{distribution.lower()}'):
    os.mkdir(f'../forecasts_probNN_{distribution.lower()}')

if not os.path.exists(f'../distparams_probNN_{distribution.lower()}'):
    os.mkdir(f'../distparams_probNN_{distribution.lower()}')

if not os.path.exists(f'../trialfiles'):
    os.mkdir(f'../trialfiles')

print(cty, distribution)

if cty != 'DE':
    raise ValueError('Incorrect country')
if distribution not in paramcount:
    raise ValueError('Incorrect distribution')

# read data file
data = pd.read_csv(f'../Datasets/{cty}.csv', index_col=0)
data.index = [datetime.strptime(e, '%Y-%m-%d %H:%M:%S') for e in data.index]
# data = data.iloc[:4*364*24] # take the first 4 years - 1456 days

def runoneday(inp):
    params, dayno = inp
    df = data.iloc[dayno*24:dayno*24+1456*24+24]
    # prepare the input/output dataframes
    Y = np.zeros((1456, 24))
    # Yf = np.zeros((1, 24)) # no Yf for rolling prediction
    for d in range(1456):
        Y[d, :] = df.loc[df.index[d*24:(d+1)*24], 'Price'].to_numpy()
    Y = Y[7:, :] # skip first 7 days
    # for d in range(1):
    #     Yf[d, :] = df.loc[df.index[(d+1092)*24:(d+1093)*24], 'Price'].to_numpy()
    X = np.zeros((1456+1, INP_SIZE))
    for d in range(7, 1456+1):
        X[d, :24] = df.loc[df.index[(d-1)*24:(d)*24], 'Price'].to_numpy() # D-1 price
        X[d, 24:48] = df.loc[df.index[(d-2)*24:(d-1)*24], 'Price'].to_numpy() # D-2 price
        X[d, 48:72] = df.loc[df.index[(d-3)*24:(d-2)*24], 'Price'].to_numpy() # D-3 price
        X[d, 72:96] = df.loc[df.index[(d-7)*24:(d-6)*24], 'Price'].to_numpy() # D-7 price
        X[d, 96:120] = df.loc[df.index[(d)*24:(d+1)*24], df.columns[1]].to_numpy() # D load forecast
        X[d, 120:144] = df.loc[df.index[(d-1)*24:(d)*24], df.columns[1]].to_numpy() # D-1 load forecast
        X[d, 144:168] = df.loc[df.index[(d-7)*24:(d-6)*24], df.columns[1]].to_numpy() # D-7 load forecast
        X[d, 168:192] = df.loc[df.index[(d)*24:(d+1)*24], df.columns[2]].to_numpy() # D RES sum forecast
        X[d, 192:216] = df.loc[df.index[(d-1)*24:(d)*24], df.columns[2]].to_numpy() # D-1 RES sum forecast
        # #original author's code
        # X[d, 216] = df.loc[df.index[(d-2)*24:(d-1)*24:24], df.columns[3]].to_numpy() # D-2 EUA
        # X[d, 217] = df.loc[df.index[(d-2)*24:(d-1)*24:24], df.columns[4]].to_numpy() # D-2 API2_Coal
        # X[d, 218] = df.loc[df.index[(d-2)*24:(d-1)*24:24], df.columns[5]].to_numpy() # D-2 TTF_Gas
        # X[d, 219] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[6]].to_numpy() # D-2 Brent oil
        X[d, 216] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[3]].to_numpy().item()
        X[d, 217] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[4]].to_numpy().item()
        X[d, 218] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[5]].to_numpy().item()
        X[d, 219] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[6]].to_numpy().item()
        # original author's code
        X[d, 220] = data.index[d].weekday()
    
    # '''
    # input feature selection
    colmask = [False] * INP_SIZE
    if params['price_D-1']:
        colmask[:24] = [True] * 24
    if params['price_D-2']:
        colmask[24:48] = [True] * 24
    if params['price_D-3']:
        colmask[48:72] = [True] * 24
    if params['price_D-7']:
        colmask[72:96] = [True] * 24
    if params['load_D']:
        colmask[96:120] = [True] * 24
    if params['load_D-1']:
        colmask[120:144] = [True] * 24
    if params['load_D-7']:
        colmask[144:168] = [True] * 24
    if params['RES_D']:
        colmask[168:192] = [True] * 24
    if params['RES_D-1']:
        colmask[192:216] = [True] * 24
    if params['EUA']:
        colmask[216] = True
    if params['Coal']:
        colmask[217] = True
    if params['Gas']:
        colmask[218] = True
    if params['Oil']:
        colmask[219] = True
    if params['Dummy']:
        colmask[220] = True
    X = X[:, colmask]
    # '''
    Xf = X[-1:, :]
    X = X[7:-1, :]
    # begin building a model
    inputs = keras.Input(X.shape[1]) # <= INP_SIZE as some columns might have been turned off
    # batch normalization
    batchnorm = True#params['batch_normalization'] # trial.suggest_categorical('batch_normalization', [True, False])
    if batchnorm:
        norm = keras.layers.BatchNormalization()(inputs)
        last_layer = norm
    else:
        last_layer = inputs
    # dropout
    dropout = params['dropout'] # trial.suggest_categorical('dropout', [True, False])
    if dropout:
        rate = params['dropout_rate'] # trial.suggest_float('dropout_rate', 0, 1)
        drop = keras.layers.Dropout(rate)(last_layer)
        last_layer = drop
    # regularization of 1st hidden layer, 
    #activation - output, kernel - weights/parameters of input
    regularize_h1_activation = params['regularize_h1_activation']
    regularize_h1_kernel = params['regularize_h1_kernel']
    h1_activation_rate = (0.0 if not regularize_h1_activation 
                          else params['h1_activation_rate_l1'])
    h1_kernel_rate = (0.0 if not regularize_h1_kernel 
                      else params['h1_kernel_rate_l1'])
    # define 1st hidden layer with regularization
    hidden = keras.layers.Dense(params['neurons_1'], 
                                activation=params['activation_1'],
                                # kernel_initializer='ones',
                                kernel_regularizer=keras.regularizers.L1(h1_kernel_rate),
                                activity_regularizer=keras.regularizers.L1(h1_activation_rate))(last_layer)
    # regularization of 2nd hidden layer, 
    #activation - output, kernel - weights/parameters of input
    regularize_h2_activation = params['regularize_h2_activation']
    regularize_h2_kernel = params['regularize_h2_kernel']
    h2_activation_rate = (0.0 if not regularize_h2_activation 
                          else params['h2_activation_rate_l1'])
    h2_kernel_rate = (0.0 if not regularize_h2_kernel 
                      else params['h2_kernel_rate_l1'])
    # define 2nd hidden layer with regularization
    hidden = keras.layers.Dense(params['neurons_2'], 
                                activation=params['activation_2'],
                                # kernel_initializer='ones',
                                kernel_regularizer=keras.regularizers.L1(h2_kernel_rate),
                                activity_regularizer=keras.regularizers.L1(h2_activation_rate))(hidden)
    if paramcount[distribution] is None:
        outputs = keras.layers.Dense(24, activation='linear')(hidden)
        model = keras.Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=keras.optimizers.Adam(params['learning_rate']),
                      loss='mae',
                      metrics='mae')
    else:
        # now define parameter layers with their regularization
        param_layers = []
        param_names = ["loc", "scale", "tailweight", "skewness"]
        for p in range(paramcount[distribution]):
            regularize_param_kernel = params['regularize_'+param_names[p]]
            param_kernel_rate = (0.0 if not regularize_param_kernel 
                                 else params[str(param_names[p])+'_rate_l1'])
            param_layers.append(keras.layers.Dense(
                24, activation='linear', # kernel_initializer='ones',
                kernel_regularizer=keras.regularizers.L1(param_kernel_rate))(hidden))
        # concatenate the parameter layers to one
        linear = tf.keras.layers.concatenate(param_layers)
        # define outputs
        if distribution == 'Normal':
            outputs = tfp.layers.DistributionLambda(
                    lambda t: tfd.Normal(
                        loc=t[..., :24],
                        scale = 1e-3 + 3 * tf.math.softplus(t[..., 24:])))(linear)
        elif distribution == 'StudentT':
            outputs = tfp.layers.DistributionLambda(
                    lambda t: tfd.StudentT(
                        loc=t[..., :24],
                        scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                        df=1 + 3 * tf.math.softplus(t[..., 48:])))(linear)
        elif distribution == 'JSU':
            outputs = tfp.layers.DistributionLambda(
                    lambda t: tfd.JohnsonSU(
                        loc=t[..., :24],
                        scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                        tailweight= 1 + 3 * tf.math.softplus(t[..., 48:72]),
                        skewness=t[..., 72:]))(linear)
        elif distribution == 'SinhArcsinh':
            outputs = tfp.layers.DistributionLambda(
                    lambda t: tfd.SinhArcsinh(
                        loc=t[..., :24],
                        scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                        tailweight=1e-3 + 3 * tf.math.softplus(t[..., 48:72]),
                        skewness=t[..., 72:]))(linear)
        elif distribution == 'NormalInverseGaussian':
            outputs = tfp.layers.DistributionLambda(
                    lambda t: tfd.NormalInverseGaussian(
                        loc=t[..., :24],
                        scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                        tailweight=1e-3 + 3 * tf.math.softplus(t[..., 48:72]),
                        skewness=t[..., 72:]))(linear) 
        else:
            raise ValueError(f'Incorrect distribution {distribution}')
        model = keras.Model(inputs = inputs, outputs=outputs)
        model.compile(optimizer=keras.optimizers.Adam(params['learning_rate']),
                      loss=lambda y, rv_y: -rv_y.log_prob(y),
                      metrics='mae')
    # '''
    # define callbacks
    callbacks = [keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)]
    perm = np.random.permutation(np.arange(X.shape[0]))
    VAL_DATA = .2
    trainsubset = perm[:int((1 - VAL_DATA)*len(perm))]
    valsubset = perm[int((1 - VAL_DATA)*len(perm)):]
    model.fit(X[trainsubset], Y[trainsubset], epochs=1500, validation_data=(X[valsubset], Y[valsubset]), callbacks=callbacks, batch_size=32, verbose=False)
    
    # metrics = model.evaluate(Xf, Yf) # for point its a list of one [loss, MAE]
    # we optimize the returned value, -1 will always take the model with best MAE

    # pred = model.predict(Xf)[0]
    if paramcount[distribution] is not None:
        dist = model(Xf)
        if distribution == 'Normal':
            getters = {'loc': dist.loc, 'scale': dist.scale}
        elif distribution == 'StudentT':
            getters = {'loc': dist.loc, 'scale': dist.scale, 'df': dist.df}
        elif distribution in {'JSU', 'SinhArcsinh', 'NormalInverseGaussian'}:
            getters = {'loc': dist.loc, 'scale': dist.scale, 
                       'tailweight': dist.tailweight, 'skewness': dist.skewness}
        print(getters)
        params = {k: [float(e) for e in v.numpy()[0]] for k, v in getters.items()}
        print(params)
        json.dump(params, open(os.path.join(f'../distparams_probNN_{distribution.lower()}', datetime.strftime(df.index[-24], '%Y-%m-%d')), 'w'))
        pred = model.predict(np.tile(Xf, (10000, 1)))
        predDF = pd.DataFrame(index=df.index[-24:])
        predDF['real'] = df.loc[df.index[-24:], 'Price'].to_numpy()
        predDF['forecast'] = pd.NA
        predDF.loc[predDF.index[:], 'forecast'] = pred.mean(0)
        # predDF.to_csv(os.path.join('../forecasts', datetime.strftime(df.index[-24], '%Y-%m-%d')))
        np.savetxt(os.path.join(f'../forecasts_probNN_{distribution.lower()}', datetime.strftime(df.index[-24], '%Y-%m-%d')), pred, delimiter=',', fmt='%.3f')
    else:
        predDF = pd.DataFrame(index=df.index[-24:])
        predDF['real'] = df.loc[df.index[-24:], 'Price'].to_numpy()
        predDF['forecast'] = pd.NA
        predDF.loc[predDF.index[:], 'forecast'] = model.predict(Xf)[0]
        pred = model.predict(Xf)
        np.savetxt(os.path.join(f'../forecasts_probNN_{distribution.lower()}', datetime.strftime(df.index[-24], '%Y-%m-%d')), pred, delimiter=',', fmt='%.3f')
    print(predDF)
    return predDF




In [ ]:
# chatgpt code
distribution = "Normal"  
study_file = os.path.abspath(f"../trialfiles/FINAL_DE_selection_prob_{distribution.lower()}")  # e.g. .../FINAL_DE_selection_prob_normal
storage_url = f"sqlite:///{study_file}"

summaries = optuna.study.get_all_study_summaries(storage_url)
if not summaries:
    raise RuntimeError(f"No Optuna studies found inside {study_file}")

study_name = summaries[0].study_name
study = optuna.load_study(study_name=study_name, storage=storage_url)
print("Loaded:", study_name, "| trials:", len(study.trials))

required = {
    "dropout", "learning_rate", "neurons_1", "neurons_2", "activation_1", "activation_2",
    "price_D-1","price_D-2","price_D-3","price_D-7",
    "load_D","load_D-1","load_D-7","RES_D","RES_D-1",
    "EUA","Coal","Gas","Oil","Dummy",
    "regularize_h1_activation","regularize_h1_kernel","regularize_h2_activation","regularize_h2_kernel",
    "regularize_loc","regularize_scale",
}

def usable(t):
    if t.state not in (TrialState.FAIL, TrialState.COMPLETE):
        return False
    if not t.params:
        return False
    return required.issubset(set(t.params.keys()))

cands = [t for t in study.trials if usable(t)]
print("Usable trials:", [t.number for t in cands[:10]])

if not cands:
    richest = max(study.trials, key=lambda t: len(t.params))
    missing = sorted(list(required - set(richest.params.keys())))
    raise RuntimeError(f"No usable trials found. Richest trial={richest.number} missing keys={missing[:20]}")

trial = cands[0]  
params = trial.params
print("Using trial:", trial.number, "state:", trial.state, "n_params:", len(params))

_ = runoneday((params, 0))

# # author's code
# optuna.logging.get_logger('optuna').addHandler(logging.StreamHandler(sys.stdout))
# study_name = f'FINAL_DE_selection_prob_{distribution.lower()}' # 'on_new_data_no_feature_selection'
# storage_name = f'sqlite:///../trialfiles/{study_name}'
# study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
# print(study.trials_dataframe())

In [ ]:
# diagnostics cell
print("Storage: ", storage_name)
print("Study: ", study.study_name)
print("n_trials: ", len(study.trials))
completed = study.get_trials(states=(TrialState.COMPLETE,))
print("n_completed: ", len(completed))
print(study.trials_dataframe().head())
print(study.trials_dataframe()["state"].value_counts() if len(study.trials) else "no trials")

best_params = study.best_params
print(best_params)

inputlist = [(best_params, day) for day in range(len(data) // 24 - 1456)]
print(len(inputlist))

# for e in inputlist:
#     _ = runoneday(e)

with Pool(max(os.cpu_count() // 4, 1)) as p:
    _ = p.map(runoneday, inputlist)


In [ ]:
# Accepts arguments:
#     cty (currently only DE), default: DE
#     distribution (Normal, StudentT, JSU, SinhArcsinh and NormalInverseGaussian), default: Normal

distribution = 'Normal'
paramcount = {'Normal': 2,
              'StudentT': 3,
              'JSU': 4,
              'SinhArcsinh': 4,
              'NormalInverseGaussian': 4,
              'Point': None
}
val_multi = 3 # int for # of re-trains - 1 corresponds to old approach
val_window = 364 // val_multi

if not os.path.exists(f'../trialfiles'):
    os.mkdir(f'../trialfiles')

INP_SIZE = 221
activations = ['sigmoid', 'relu', 'elu', 'tanh', 'softplus', 'softmax']

binopt = [True, False]

cty = 'ES'
repo_root = Path.cwd()
storeDBintmp = False

print(cty, distribution)

if cty != 'ES':
    raise ValueError('Incorrect country')
if distribution not in paramcount:
    raise ValueError('Incorrect distribution')

# read data file
data = pd.read_csv(f'../Datasets/{cty}.csv', index_col=0)
data.index = [datetime.strptime(e, '%Y-%m-%d %H:%M:%S') for e in data.index]
data = data.iloc[:4*364*24] # take the first 4 years - 1456 days

def objective(trial):
    
    # prepare the input/output dataframes
    Y = np.zeros((1456, 24))
    Yf = np.zeros((364, 24))
    for d in range(1456):
        Y[d, :] = data.loc[data.index[d*24:(d+1)*24], 'Price'].to_numpy()
    # Y = Y[7:, :] # skip first 7 days
    for d in range(364):
        Yf[d, :] = data.loc[data.index[(d+1092)*24:(d+1093)*24], 'Price'].to_numpy()
    # 
    # build X 
    X = np.zeros((1092+364, INP_SIZE))

    for d in range(7, 1092+364):
      
        X[d, :24] = data.loc[data.index[(d-1)*24:(d)*24], 'Price'].to_numpy() # D-1 price
        X[d, 24:48] = data.loc[data.index[(d-2)*24:(d-1)*24], 'Price'].to_numpy() # D-2 price
        X[d, 48:72] = data.loc[data.index[(d-3)*24:(d-2)*24], 'Price'].to_numpy() # D-3 price
        X[d, 72:96] = data.loc[data.index[(d-7)*24:(d-6)*24], 'Price'].to_numpy() # D-7 price
        X[d, 96:120] = data.loc[data.index[(d)*24:(d+1)*24], data.columns[1]].to_numpy() # D load forecast
        X[d, 120:144] = data.loc[data.index[(d-1)*24:(d)*24], data.columns[1]].to_numpy() # D-1 load forecast
        X[d, 144:168] = data.loc[data.index[(d-7)*24:(d-6)*24], data.columns[1]].to_numpy() # D-7 load forecast
        X[d, 168:192] = data.loc[data.index[(d)*24:(d+1)*24], data.columns[2]].to_numpy() # D RES sum forecast
        X[d, 192:216] = data.loc[data.index[(d-1)*24:(d)*24], data.columns[2]].to_numpy() # D-1 RES sum forecast
        
        # # original aouthors code
        # X[d, 216] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[3]].to_numpy() # D-2 EUA
        # X[d, 217] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[4]].to_numpy() # D-2 API2_Coal
        # X[d, 218] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[5]].to_numpy() # D-2 TTF_Gas
        # X[d, 219] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[6]].to_numpy() # D-2 Brent oil
        
        X[d, 216] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[3]].to_numpy().item()
        X[d, 217] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[4]].to_numpy().item()
        X[d, 218] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[5]].to_numpy().item()
        X[d, 219] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[6]].to_numpy().item()
        
        # authors code
        X[d, 220] = data.index[d].weekday()

    # '''
    # input feature selection
    colmask = [False] * INP_SIZE
    if trial.suggest_categorical('price_D-1', binopt):
        colmask[:24] = [True] * 24
    if trial.suggest_categorical('price_D-2', binopt):
        colmask[24:48] = [True] * 24
    if trial.suggest_categorical('price_D-3', binopt):
        colmask[48:72] = [True] * 24
    if trial.suggest_categorical('price_D-7', binopt):
        colmask[72:96] = [True] * 24
    if trial.suggest_categorical('load_D', binopt):
        colmask[96:120] = [True] * 24
    if trial.suggest_categorical('load_D-1', binopt):
        colmask[120:144] = [True] * 24
    if trial.suggest_categorical('load_D-7', binopt):
        colmask[144:168] = [True] * 24
    if trial.suggest_categorical('RES_D', binopt):
        colmask[168:192] = [True] * 24
    if trial.suggest_categorical('RES_D-1', binopt):
        colmask[192:216] = [True] * 24
    if trial.suggest_categorical('EUA', binopt):
        colmask[216] = True
    if trial.suggest_categorical('Coal', binopt):
        colmask[217] = True
    if trial.suggest_categorical('Gas', binopt):
        colmask[218] = True
    if trial.suggest_categorical('Oil', binopt):
        colmask[219] = True
    if trial.suggest_categorical('Dummy', binopt):
        colmask[220] = True
    X = X[:, colmask]
    
    # '''
    Xwhole = X.copy()
    Ywhole = Y.copy()
    Yfwhole = Yf.copy()
    metrics_sub = []
    
    for train_no in range(val_multi):
        
        start = val_window * train_no
        X = Xwhole[start:1092+start, :]
        Xf = Xwhole[1092+start:1092+start+val_window, :]
        Y = Ywhole[start:1092+start, :]
        Yf = Ywhole[1092+start:1092+start+val_window, :]
        X = X[7:1092, :]
        Y = Y[7:1092, :]
        
        # begin building a model
        inputs = keras.Input(shape=(X.shape[1],)) # <= INP_SIZE as some columns might have been turned off
        # batch normalization
        # we decided to always normalize the inputs
        batchnorm = True #trial.suggest_categorical('batch_normalization', [True, False])
        if batchnorm:
            norm = keras.layers.BatchNormalization()(inputs)
            last_layer = norm
        else:
            last_layer = inputs
        # dropout
        dropout = trial.suggest_categorical('dropout', binopt)
        if dropout:
            rate = trial.suggest_float('dropout_rate', 0, 1)
            drop = keras.layers.Dropout(rate)(last_layer)
            last_layer = drop
        # regularization of 1st hidden layer,
        #activation - output, kernel - weights/parameters of input
        regularize_h1_activation = trial.suggest_categorical('regularize_h1_activation', binopt)
        regularize_h1_kernel = trial.suggest_categorical('regularize_h1_kernel', binopt)
        h1_activation_rate = (0.0 if not regularize_h1_activation
                              else trial.suggest_float('h1_activation_rate_l1', 1e-5, 1e-2, log=True))
        h1_kernel_rate = (0.0 if not regularize_h1_kernel
                          else trial.suggest_float('h1_kernel_rate_l1', 1e-5, 1e-2, log=True))
        # define 1st hidden layer with regularization
        hidden = keras.layers.Dense(trial.suggest_int('neurons_1', 256, 512, log=False),
                                    activation=trial.suggest_categorical('activation_1', activations),
                                    # kernel_initializer='ones',
                                    kernel_regularizer=keras.regularizers.L1(h1_kernel_rate),
                                    activity_regularizer=keras.regularizers.L1(h1_activation_rate))(last_layer)
        # regularization of 2nd hidden layer,
        #activation - output, kernel - weights/parameters of input
        regularize_h2_activation = trial.suggest_categorical('regularize_h2_activation', binopt)
        regularize_h2_kernel = trial.suggest_categorical('regularize_h2_kernel', binopt)
        h2_activation_rate = (0.0 if not regularize_h2_activation
                              else trial.suggest_float('h2_activation_rate_l1', 1e-5, 1e-2, log=True))
        h2_kernel_rate = (0.0 if not regularize_h2_kernel
                          else trial.suggest_float('h2_kernel_rate_l1', 1e-5, 1e-2, log=True))
        # define 2nd hidden layer with regularization
        hidden = keras.layers.Dense(trial.suggest_int('neurons_2', 256, 512, log=False),
                                    activation=trial.suggest_categorical('activation_2', activations),
                                    # kernel_initializer='ones',
                                    kernel_regularizer=keras.regularizers.L1(h2_kernel_rate),
                                    activity_regularizer=keras.regularizers.L1(h2_activation_rate))(hidden)
        if paramcount[distribution] is None:
            outputs = tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., :24],scale=1e-3 + 3.0 * tf.nn.softplus(t[..., 24:])))(linear)
            # # authors one
            # outputs = keras.layers.Dense(24, activation='linear')(hidden)
            
            model = keras.Model(inputs=inputs, outputs=outputs)
            model.compile(optimizer=keras.optimizers.legacy.Adam(trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)),
                          # loss=lambda y, rv_y: -rv_y.log_prob(y),
                          loss='mae',
                          metrics='mae')
        else:
            # now define parameter layers with their regularization
            param_layers = []
            param_names = ["loc", "scale", "tailweight", "skewness"]
            for p in range(paramcount[distribution]):
                # regularize_param_kernel = True
                # param_kernel_rate = 0.1
                regularize_param_kernel = trial.suggest_categorical('regularize_'+param_names[p], binopt)
                param_kernel_rate = (0.0 if not regularize_param_kernel
                                     else trial.suggest_float(param_names[p]+'_rate_l1', 1e-5, 1e-2, log=True))
                param_layers.append(keras.layers.Dense(
                    24, activation='linear', # kernel_initializer='ones',
                    kernel_regularizer=keras.regularizers.L1(param_kernel_rate))(hidden))
            # concatenate the parameter layers to one
            linear = tf.keras.layers.concatenate(param_layers)
            # define outputs
            if distribution == 'Normal':
                outputs = tfp.layers.DistributionLambda(
                        lambda t: tfd.Normal(
                            loc=t[..., :24],
                            scale = 1e-3 + 3 * tf.math.softplus(t[..., 24:])))(linear)
            elif distribution == 'StudentT':
                outputs = tfp.layers.DistributionLambda(
                        lambda t: tfd.StudentT(
                            loc=t[..., :24],
                            scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                            df=1 + 3 * tf.math.softplus(t[..., 48:])))(linear)
            elif distribution == 'JSU':
                outputs = tfp.layers.DistributionLambda(
                        lambda t: tfd.JohnsonSU(
                            loc=t[..., :24],
                            scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                            tailweight= 1 + 3 * tf.math.softplus(t[..., 48:72]),
                            skewness=t[..., 72:]))(linear)
            elif distribution == 'SinhArcsinh':
                outputs = tfp.layers.DistributionLambda(
                        lambda t: tfd.SinhArcsinh(
                            loc=t[..., :24],
                            scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                            tailweight=1e-3 + 3 * tf.math.softplus(t[..., 48:72]),
                            skewness=t[..., 72:]))(linear)
            elif distribution == 'NormalInverseGaussian':
                outputs = tfp.layers.DistributionLambda(
                        lambda t: tfd.NormalInverseGaussian(
                            loc=t[..., :24],
                            scale=1e-3 + 3 * tf.math.softplus(t[..., 24:48]),
                            tailweight=1e-3 + 3 * tf.math.softplus(t[..., 48:72]),
                            skewness=t[..., 72:]))(linear)
            else:
                raise ValueError(f'Incorrect distribution {distribution}')
            model = keras.Model(inputs = inputs, outputs=outputs)
            model.compile(optimizer=keras.optimizers.legacy.Adam(trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)),
                          loss=lambda y, rv_y: -rv_y.log_prob(y),
                          metrics='mae')
        # '''

        # define callbacks
        callbacks = [keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)]    
        model.fit(X, Y, epochs=300, validation_data=(Xf, Yf), callbacks=callbacks, batch_size=256, verbose=2)

        metrics = model.evaluate(Xf, Yf, verbose=0) # for point its a list of one [loss, MAE] as 
        metrics_sub.append(metrics[0])
        # we optimize the returned value, -1 will always take the model with best MAE
    return np.mean(metrics_sub)

#optuna.logging.get_logger('optuna').addHandler(logging.StreamHandler(sys.stdout))
optuna.logging.set_verbosity(optuna.logging.WARNING)
study_name = f'FINAL_DE_selection_prob_{distribution.lower()}'
if storeDBintmp:
    storage_name = f'sqlite:////tmp/{study_name}'
else:
    storage_name = f'sqlite:///../trialfiles/{study_name}'
# below calls to create either random sampler and default (tree parzen estimator)
# study can be resumed using a different one
# (or one process can sample randomly, the second one run the TPE sampler at the same time)
# study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, sampler=optuna.samplers.RandomSampler())
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
study.optimize(objective, n_trials=4, show_progress_bar=True)
best_params = study.best_params
print(best_params)
print(study.trials_dataframe())
if storeDBintmp:
    print('Trials DB stored in /tmp')


### MIssing days

In [55]:
df_at = pd.read_csv("/Users/bekzodamonov/Documents/GitHub/distributionalnn-execute-once/Datasets/AT.csv",index_col=0)
df_at.index = pd.to_datetime(df_at.index, format="%Y-%m-%d %H:%M:%S", errors="raise")
df_at = df_at.sort_index()
idx = pd.DatetimeIndex(df_at.index)

print("rows:", len(df_at))
print("range:", idx.min(), "->", idx.max())
print("duplicates:", idx.has_duplicates)
print("sorted:", idx.is_monotonic_increasing)

# ---- expected full hourly timeline between min and max ----
expected = pd.date_range(start=idx.min(), end=idx.max(), freq="h")

missing_ts = expected.difference(idx)
print("\n=== Overall missing hours ===")
print("expected hours:", len(expected))
print("actual hours:  ", len(idx))
print("missing hours: ", len(missing_ts))

# ---- day-level completeness (calendar days) ----
actual_per_day = pd.Series(1, index=idx).groupby(idx.floor("D")).sum()
expected_per_day = pd.Series(1, index=expected).groupby(expected.floor("D")).sum()

day_stats = pd.DataFrame({
    "actual_hours": actual_per_day,
    "expected_hours": expected_per_day
}).fillna(0).astype(int)

day_stats["missing_hours"] = day_stats["expected_hours"] - day_stats["actual_hours"]
incomplete_days = day_stats[day_stats["missing_hours"] != 0].sort_values("missing_hours", ascending=False)

print("\n=== Day-level ===")
print("days in range:", len(day_stats))
print("incomplete days:", len(incomplete_days))

missing_days = day_stats[day_stats["actual_hours"] == 0]
print("\n=== Missing days entirely (0 hours present) ===")
print("count:", len(missing_days))

# ---- frequency sanity (what time jumps exist) ----
diff_counts = pd.Series(idx).diff().value_counts().head(15)
print("\n=== Top time differences in ACTUAL data ===")
print(diff_counts)

day_stats["year"] = day_stats.index.year
year_summary = (
    day_stats.groupby("year").agg(
        days=("missing_hours", "size"),
        incomplete_days=("missing_hours", lambda s: (s != 0).sum()),
        missing_hours_total=("missing_hours", "sum")))

year_summary["incomplete_days_share"] = year_summary["incomplete_days"] / year_summary["days"]

print("\n=== Year-level summary ===")
print(year_summary)



rows: 96908
range: 2014-12-09 23:00:00 -> 2026-01-01 00:00:00
duplicates: False
sorted: True

=== Overall missing hours ===
expected hours: 96962
actual hours:   96908
missing hours:  54

=== Day-level ===
days in range: 4042
incomplete days: 8

=== Missing days entirely (0 hours present) ===
count: 0

=== Top time differences in ACTUAL data ===
time_utc
0 days 01:00:00    96902
1 days 01:00:00        2
0 days 04:00:00        1
0 days 03:00:00        1
0 days 02:00:00        1
Name: count, dtype: int64

=== Year-level summary ===
      days  incomplete_days  missing_hours_total  incomplete_days_share
year                                                                   
2014    23                0                    0               0.000000
2015   365                6                   51               0.016438
2016   366                0                    0               0.000000
2017   365                0                    0               0.000000
2018   365                2     

In [ ]:
val_multi = 13 # int for # of re-trains - 1 corresponds to old approach
val_window = 364 // val_multi

if not os.path.exists(f'../trialfiles'):
    os.mkdir(f'../trialfiles')

INP_SIZE = 221
activations = ['sigmoid', 'relu', 'elu', 'tanh', 'softplus', 'softmax']

binopt = [True, False]

cty = 'ES'

# read data file
repo_root = Path.cwd()
data = pd.read_csv( f"../Datasets/AT.csv", index_col=0)
data.index = [datetime.strptime(e, '%Y-%m-%d %H:%M:%S') for e in data.index]
data = data.iloc[:4*364*24] # take the first 4 years - 1456 days


# prepare the input/output dataframes
Y = np.zeros((1456, 24))
Yf = np.zeros((364, 24))
for d in range(1456):
    Y[d, :] = data.loc[data.index[d*24:(d+1)*24], 'Price'].to_numpy()
# Y = Y[7:, :] # skip first 7 days
for d in range(364):
    Yf[d, :] = data.loc[data.index[(d+1092)*24:(d+1093)*24], 'Price'].to_numpy()
# 
X = np.zeros((1092+364, INP_SIZE))
for d in range(7, 1092+364):
    X[d, :24] = data.loc[data.index[(d-1)*24:(d)*24], 'Price'].to_numpy() # D-1 price
    X[d, 24:48] = data.loc[data.index[(d-2)*24:(d-1)*24], 'Price'].to_numpy() # D-2 price
    X[d, 48:72] = data.loc[data.index[(d-3)*24:(d-2)*24], 'Price'].to_numpy() # D-3 price
    X[d, 72:96] = data.loc[data.index[(d-7)*24:(d-6)*24], 'Price'].to_numpy() # D-7 price
    X[d, 96:120] = data.loc[data.index[(d)*24:(d+1)*24], data.columns[1]].to_numpy() # D load forecast
    X[d, 120:144] = data.loc[data.index[(d-1)*24:(d)*24], data.columns[1]].to_numpy() # D-1 load forecast
    X[d, 144:168] = data.loc[data.index[(d-7)*24:(d-6)*24], data.columns[1]].to_numpy() # D-7 load forecast
    X[d, 168:192] = data.loc[data.index[(d)*24:(d+1)*24], data.columns[2]].to_numpy() # D RES sum forecast
    X[d, 192:216] = data.loc[data.index[(d-1)*24:(d)*24], data.columns[2]].to_numpy()[0] # D-1 RES sum forecast
    X[d, 216] = data.loc[data.index[(d-2)*24:(d-1)*24:24], data.columns[3]].to_numpy()[0] # D-2 EUA





In [105]:
repo_root = Path(__file__).resolve().parent.parent

NameError: name '__file__' is not defined

In [54]:
df = pd.read_csv("/Users/bekzodamonov/Documents/GitHub/distributionalnn-execute-once/Datasets/ES.csv",index_col=0)
df.index = pd.to_datetime(df.index, format="%Y-%m-%d %H:%M:%S", errors="raise")
df = df.sort_index()
idx = pd.DatetimeIndex(df.index)

print("rows:", len(df))
print("range:", idx.min(), "->", idx.max())
print("duplicates:", idx.has_duplicates)
print("sorted:", idx.is_monotonic_increasing)

# ---- expected full hourly timeline between min and max ----
expected = pd.date_range(start=idx.min(), end=idx.max(), freq="h")

missing_ts = expected.difference(idx)
print("\n=== Overall missing hours ===")
print("expected hours:", len(expected))
print("actual hours:  ", len(idx))
print("missing hours: ", len(missing_ts))

# ---- day-level completeness (calendar days) ----
actual_per_day = pd.Series(1, index=idx).groupby(idx.floor("D")).sum()
expected_per_day = pd.Series(1, index=expected).groupby(expected.floor("D")).sum()

day_stats = pd.DataFrame({
    "actual_hours": actual_per_day,
    "expected_hours": expected_per_day
}).fillna(0).astype(int)

day_stats["missing_hours"] = day_stats["expected_hours"] - day_stats["actual_hours"]
incomplete_days = day_stats[day_stats["missing_hours"] != 0].sort_values("missing_hours", ascending=False)

print("\n=== Day-level ===")
print("days in range:", len(day_stats))
print("incomplete days:", len(incomplete_days))

missing_days = day_stats[day_stats["actual_hours"] == 0]
print("\n=== Missing days entirely (0 hours present) ===")
print("count:", len(missing_days))

# ---- frequency sanity (what time jumps exist) ----
diff_counts = pd.Series(idx).diff().value_counts().head(15)
print("\n=== Top time differences in ACTUAL data ===")
print(diff_counts)

day_stats["year"] = day_stats.index.year
year_summary = (
    day_stats.groupby("year").agg(
        days=("missing_hours", "size"),
        incomplete_days=("missing_hours", lambda s: (s != 0).sum()),
        missing_hours_total=("missing_hours", "sum")))

year_summary["incomplete_days_share"] = year_summary["incomplete_days"] / year_summary["days"]

print("\n=== Year-level summary ===")
print(year_summary)


rows: 92303
range: 2014-12-10 23:00:00 -> 2026-01-01 00:00:00
duplicates: False
sorted: True

=== Overall missing hours ===
expected hours: 96938
actual hours:   92303
missing hours:  4635

=== Day-level ===
days in range: 4041
incomplete days: 2285

=== Missing days entirely (0 hours present) ===
count: 0

=== Top time differences in ACTUAL data ===
time_utc
0 days 01:00:00    88865
0 days 02:00:00     2781
0 days 03:00:00      419
0 days 04:00:00      108
0 days 05:00:00       53
0 days 06:00:00       42
0 days 07:00:00       15
0 days 08:00:00        8
0 days 09:00:00        3
0 days 10:00:00        2
0 days 11:00:00        2
0 days 18:00:00        1
0 days 13:00:00        1
0 days 16:00:00        1
0 days 19:00:00        1
Name: count, dtype: int64

=== Year-level summary ===
      days  incomplete_days  missing_hours_total  incomplete_days_share
year                                                                   
2014    22               16                   30               0.

In [48]:
day_stats["year"] = day_stats.index.year
year_summary = (
    day_stats.groupby("year").agg(
        days=("missing_hours", "size"),
        incomplete_days=("missing_hours", lambda s: (s != 0).sum()),
        missing_hours_total=("missing_hours", "sum")))

year_summary["incomplete_days_share"] = year_summary["incomplete_days"] / year_summary["days"]

print("\n=== Year-level summary ===")
print(year_summary)

# show the most incomplete days within each year
TOP_K = 15
top_days_each_year = (
    day_stats[day_stats["missing_hours"] > 0]
    .sort_values(["year", "missing_hours"], ascending=[True, False])
    .groupby("year")
    .head(TOP_K)
    .drop(columns=["year"])
)

print(f"\n=== Top {TOP_K} most incomplete days PER YEAR ===")
for y, block in top_days_each_year.groupby(top_days_each_year.index.year):
    print(f"\n--- {y} ---")
    print(block)



=== Year-level summary ===
      days  incomplete_days  missing_hours_total  incomplete_days_share
year                                                                   
2014    22               16                   30               0.727273
2015   365              224                  388               0.613699
2016   366              226                  417               0.617486
2017   365              243                  408               0.665753
2018   365              122                  210               0.334247
2019   365              182                  318               0.498630
2020   366              194                  332               0.530055
2021   365              198                  375               0.542466
2022   365              204                  330               0.558904
2023   365              224                  535               0.613699
2024   366              260                  856               0.710383
2025   365              192         

In [50]:
import pandas as pd
from sqlalchemy import inspect

# ---------- Helper functions (robust) ----------
def get_spec(engine) -> pd.DataFrame:
    return pd.read_sql_query("SELECT * FROM spec", engine)

def _pick_first_existing_col(df: pd.DataFrame, candidates: list[str]) -> str:
    cols = set(df.columns)
    for c in candidates:
        if c in cols:
            return c
    raise KeyError(f"None of these columns exist in spec: {candidates}\nAvailable columns: {sorted(df.columns)}")

def get_market_divisions(spec_df: pd.DataFrame, which: str = "out"):
    """
    Return unique market division codes.
    Tries the common ENTSOE schema columns and picks what exists.
    """
    if which.lower() == "out":
        col = _pick_first_existing_col(spec_df, ["OutMapTypeCode", "MapTypeCode", "out_map_type_code"])
    elif which.lower() == "in":
        col = _pick_first_existing_col(spec_df, ["InMapTypeCode", "MapTypeCode", "in_map_type_code"])
    else:
        col = _pick_first_existing_col(spec_df, ["OutMapTypeCode", "InMapTypeCode", "MapTypeCode"])
    return spec_df[col].dropna().unique()

def get_map_codes(spec_df: pd.DataFrame, which: str = "out"):
    """
    Return unique map codes.
    In your pipeline you filter on OutMapCode, so default is 'out'.
    """
    if which.lower() == "out":
        col = _pick_first_existing_col(spec_df, ["OutMapCode", "MapCode", "out_map_code"])
    elif which.lower() == "in":
        col = _pick_first_existing_col(spec_df, ["InMapCode", "MapCode", "in_map_code"])
    else:
        col = _pick_first_existing_col(spec_df, ["OutMapCode", "InMapCode", "MapCode"])
    return spec_df[col].dropna().unique()

def get_map_codes_starting_with(spec_df: pd.DataFrame, prefix: str, which: str = "out"):
    col = _pick_first_existing_col(spec_df, ["OutMapCode", "MapCode", "InMapCode"])
    s = spec_df[col].fillna("").astype(str)
    return spec_df.loc[s.str.startswith(prefix), col].dropna().unique()

def get_resolution_codes(spec_df: pd.DataFrame):
    col = _pick_first_existing_col(spec_df, ["ResolutionCode", "resolution", "resolution_code"])
    return spec_df[col].dropna().unique()

def search_mapcodes(spec: pd.DataFrame, prefix: str | None = None, contains: str | None = None, which: str = "out") -> list[str]:
    if prefix:
        codes = list(get_map_codes_starting_with(spec, prefix, which=which))
    else:
        codes = list(get_map_codes(spec, which=which))
    if contains:
        contains_u = contains.upper()
        codes = [c for c in codes if contains_u in str(c).upper()]
    return sorted(pd.unique(codes))

# ---------- Usage ----------
engine = create_entsoe_engine()
spec = get_spec(engine)

print("Spec columns:\n", sorted(spec.columns))

print("\nMarket divisions (OutMapTypeCode preferred):", sorted(get_market_divisions(spec, which="out")))
print("Resolution codes:", sorted(get_resolution_codes(spec)))

all_mapcodes = sorted(get_map_codes(spec, which="out"))
print(f"\nTotal OutMapCodes: {len(all_mapcodes)}")
print("First 50 OutMapCodes:", all_mapcodes[:50])

# "Upload" your desired codes here (edit these lists)
COUNTRY_CODES = ["DE"]     # used only to search candidates
MAPCODES      = ["DE_LU"]  # set to whatever you decide (OutMapCode)

candidates = sorted(set().union(*[set(search_mapcodes(spec, contains=cc, which="out")) for cc in COUNTRY_CODES]))
print(f"\nCandidates (OutMapCode) containing any of {COUNTRY_CODES}:")
print(candidates[:200])

missing = [c for c in MAPCODES if c not in all_mapcodes]
if missing:
    print("\n[!] These MAPCODES are not present in OutMapCode:", missing)
    print("Try: search_mapcodes(spec, contains='DE') or search_mapcodes(spec, prefix='10Y')")
else:
    print("\n[OK] All MAPCODES exist in OutMapCode:", MAPCODES)

# Preview relevant series for those codes
preview_cols = [c for c in ["Name","Type","OutMapCode","OutMapTypeCode","ResolutionCode","TimeSeriesID"] if c in spec.columns]

preview = spec[
    spec[_pick_first_existing_col(spec, ["OutMapCode","MapCode"])].isin(MAPCODES) &
    spec[_pick_first_existing_col(spec, ["OutMapTypeCode","MapTypeCode"])].astype(str).str.contains(r"\bBZN\b", na=False) &
    spec[_pick_first_existing_col(spec, ["Name"])].isin(["Prices","Load","Generation"]) &
    spec[_pick_first_existing_col(spec, ["Type"])].isin(["DayAhead","Actual"])
][preview_cols].drop_duplicates()

print("\nPreview of matching series (first 50 rows):")
print(preview.head(50))


Spec columns:
 ['Currency', 'FileName', 'InAreaCode', 'InMapCode', 'InMapTypeCode', 'Name', 'OutAreaCode', 'OutMapCode', 'OutMapTypeCode', 'ProductionType', 'ResolutionCode', 'Specification', 'TimeSeriesID', 'Type', 'Unit', 'UnitID', 'UnitName', 'spec_hash']

Market divisions (OutMapTypeCode preferred): ['BZA, BZN, CTA, CTY, LFA, MBA, SCA', 'BZA, CTA, LFA, SCA', 'BZN', 'BZN, CTA, CTY', 'BZN, CTA, CTY, IBA, IPA, MBA, SCA', 'BZN, CTA, CTY, IPA, LFA, LFB, MBA, SCA', 'BZN, CTA, CTY, LFA, LFB, MBA', 'BZN, CTA, CTY, LFA, LFB, MBA, SCA', 'BZN, CTA, CTY, LFA, MBA, SCA', 'BZN, CTA, CTY, MBA, SCA', 'BZN, CTA, LFA, LFB, MBA, SCA', 'BZN, CTA, LFA, LFB, MBA, SCA, SNA', 'BZN, CTA, LFA, MBA, SCA', 'BZN, CTA, MBA, SCA', 'BZN, CTA, SCA', 'BZN, CTY, LFB, MBA, SCA', 'BZN, IBA, IPA, LFA, MBA, SCA', 'BZN, IBA, IPA, MBA, SCA', 'BZN, IPA, MBA, SCA', 'BZN, LFB, MBA, SCA, SNA', 'BZN, MBA, SCA', 'BZN, SCA', 'CTA', 'CTA, CTY', 'CTA, CTY, LFA, LFB, MBA, SCA', 'CTA, CTY, MBA, SCA', 'CTA, LFA, SCA', 'CTA, MBA, SCA'

/var/folders/l1/26hv4ds50v70g2zr2xbmd0bm0000gn/T/ipykernel_54789/495539176.py:58: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return sorted(pd.unique(codes))


In [69]:
from sqlalchemy import create_engine
import urllib.parse

ts = "2023-11-05 19:00:00" 

def create_entsoe_engine(config=None):
    cfg = config or ENTSOE_DB
    url = (
        "mysql+pymysql://"
        f"{urllib.parse.quote_plus(cfg['user'])}:"
        f"{urllib.parse.quote_plus(cfg['password'])}@"
        f"{cfg['host']}:{cfg['port']}/{cfg['dbname']}"
    )

    return create_engine(
        url,
        pool_pre_ping=True,      
        pool_recycle=1800,       
        pool_size=5,
        max_overflow=10,
        connect_args={
            "connect_timeout": 10,
            "read_timeout": 60,
            "write_timeout": 60,},)

engine.dispose()
engine = create_entsoe_engine()
targets_price = spec[
    (spec["Name"] == "Prices") &
    (spec["Type"].isin(["DayAhead", "Actual"])) &
    (spec["OutMapCode"].isin(["ES"]))   
    # & (spec["OutMapTypeCode"].astype(str).str.contains(r"\bBZN\b", na=False))
]
ids_list = targets_price["TimeSeriesID"].dropna().astype(int).unique().tolist()
print("n price series:", len(ids_list), "first IDs:", ids_list[:10])
ids = ", ".join(map(str, ids_list))
q = f"""
SELECT TimeSeriesID, DateTime, Value
FROM vals
WHERE TimeSeriesID IN ({ids})
  AND DateTime = '{ts}'
ORDER BY TimeSeriesID
"""
print(pd.read_sql_query(q, engine))


n price series: 2 first IDs: [566, 3411]
   TimeSeriesID            DateTime  Value
0           566 2023-11-05 19:00:00   24.8


In [59]:
ts = "2023-11-05 08:00:00"

q_any = f"""
SELECT Name, Type, OutMapCode, COUNT(*) AS n
FROM vals v
JOIN spec s ON v.TimeSeriesID = s.TimeSeriesID
WHERE s.OutMapCode = 'ES'
  AND v.DateTime = '{ts}'
GROUP BY Name, Type, OutMapCode
ORDER BY n DESC;
"""
print(pd.read_sql_query(q_any, engine).head(30))


         Name      Type OutMapCode    n
0  Generation    Actual         ES  194
1  Generation  DayAhead         ES    4
2  Generation   Current         ES    2
3  Generation  Intraday         ES    2
4        Load  DayAhead         ES    1
5        Load    Actual         ES    1


In [107]:
START_YEAR      = 2020   
VAL_START_YEAR  = 2022   
TRAIN_END_YEAR  = 2023   
FINAL_END_YEAR  = 2025   

INIT_DATE_EXP   = pd.Timestamp(f'{START_YEAR}-01-01 00:00:00')     # hyper-parameter tuning: training start date
VAL_INIT_DATE   = pd.Timestamp(f'{VAL_START_YEAR}-12-28 00:00:00') # hyper-parameter tuning: training end date
TRAIN_END_DATE  = pd.Timestamp(f'{TRAIN_END_YEAR}-12-27 00:00:00') # hyper-parameter tuning: validation end date
FINAL_DATE_EXP  = pd.Timestamp(f"{FINAL_END_YEAR}-12-31 00:00:00") # rolling window: evaluation end date

train_val_days = (TRAIN_END_DATE - INIT_DATE_EXP).days
train_days  = (VAL_INIT_DATE - INIT_DATE_EXP).days
val_days   = (TRAIN_END_DATE - VAL_INIT_DATE).days
train_val_days


1456